In [1]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
import time

from tqdm.notebook import tqdm
import extruct
import pickle

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.5 Safari/605.1.15"
}

In [3]:
sitemap_url = 'https://www.nytimes.com/sitemaps/new/cooking.xml.gz'
response = requests.get(sitemap_url, headers=headers)
sitemap_xml = response.text
sitemap_soup = BeautifulSoup(sitemap_xml, features='xml')

gz_urls = []
loc_tags = sitemap_soup.find_all('loc')

for loc in loc_tags:
    gz_urls.append(loc.get_text()) 

display(gz_urls)
print(len(gz_urls))

['https://www.nytimes.com/sitemaps/new/cooking-1991-03.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2023-07.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2021-03.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2023-03.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2022-05.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2021-04.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2023-01.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2020-12.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2020-10.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2023-02.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2022-04.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2022-03.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2020-08.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2023-05.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2022-08.xml.gz',
 'https://www.nytimes.com/sitemaps/new/c

497


In [4]:
with open('../data/nytc_raw_urls.pkl', 'rb') as f:
    recipe_urls = pickle.load(f)

In [5]:
# recipe_urls = []

# for url in tqdm(gz_urls):
#     response = requests.get(url, headers=headers)
#     sitemap_soup = BeautifulSoup(response.text, features='xml')
#     loc_tags = sitemap_soup.find_all('loc')
#     for loc in loc_tags:
#         recipe_urls.append(loc.get_text()) 


In [6]:
display(recipe_urls)
len(recipe_urls)

['https://cooking.nytimes.com/recipes/1024397-crispy-potato-tacos',
 'https://cooking.nytimes.com/recipes/1024394-cashew-celery',
 'https://cooking.nytimes.com/recipes/1024372-basil-butter-pasta',
 'https://cooking.nytimes.com/recipes/1024334-turmeric-potato-salad',
 'https://cooking.nytimes.com/recipes/1024222-street-corn-pudding',
 'https://cooking.nytimes.com/recipes/1024129-gorditas-de-maiz',
 'https://cooking.nytimes.com/recipes/1024370-zucchini-tofu-udon',
 'https://cooking.nytimes.com/recipes/1024398-soy-butter-corn-ramen',
 'https://cooking.nytimes.com/recipes/1024395-salt-and-vinegar-kale-chips-with-fried-chickpeas-and-avocado',
 'https://cooking.nytimes.com/recipes/1024365-bloody-mary-tomato-salad',
 'https://cooking.nytimes.com/recipes/1024373-grilled-steak-with-sauce-rof',
 'https://cooking.nytimes.com/recipes/1024284-turkey-sandwich-with-savory-blueberry-jam',
 'https://cooking.nytimes.com/recipes/1024371-paneer-chile-dry',
 'https://cooking.nytimes.com/recipes/1024374-ove

22830

In [7]:
# with open('../data/nytc_raw_urls.pkl', 'wb') as f:
#     pickle.dump(recipe_urls, f)

Quick and dirty prune?

In [8]:
import random

random_url = random.choice(recipe_urls)
print(random_url)

https://cooking.nytimes.com/recipes/1014786-outdoor-fish-fry


In [9]:
import extruct

page = requests.get(random_url, headers=headers)
recipe_soup = BeautifulSoup(page.text, 'html.parser')

recipe_data = extruct.extract(
            page.text,
            syntaxes=['json-ld'],
            uniform=True,
        )['json-ld'][0]
display(recipe_data)

{'@context': 'http://schema.org',
 '@type': 'Recipe',
 'name': 'Outdoor Fish Fry',
 'description': 'Frying fish at home can be intimidating, not least because it’s messy and can leave the house smelling like a grease pit. Here, though, is a recipe that takes the cooking outside, to a large pan set over a propane hob or, as subsequent testing has borne out, to a large roasting pan filled with oil set over a gas grill. Simply heat the oil until it is very hot but not yet smoking, and add to it very fresh strips of fish dipped in a mixture of flour, salt, pepper and Old Bay, then into egg, and finally into panko, and fry until golden brown on each side, super-crisp, with a moist, tender and perfectly cooked interior. Buy a lot of fish. This is one of those recipes where people tend to want seconds and thirds. Serve with slaw and, if you like, tartar sauce.',
 'author': {'@type': 'Person',
  'name': 'John Willoughby And Chris Schlesinger'},
 'image': ['https://static01.nyt.com/images/2014/

In [10]:
headline = recipe_data['name'] # or 'name'
nutrition = recipe_data['nutrition']
category = recipe_data['recipeCategory']
cuisine = recipe_data['recipeCuisine']
ingredient = recipe_data['recipeIngredient']
instruction = recipe_data['recipeInstructions'][0]['text']

print(headline)
print(nutrition)
print(category)
print(cuisine)
print(ingredient)
print(instruction)

Outdoor Fish Fry
{'@context': 'http://schema.org', '@type': 'NutritionInformation', 'calories': 627, 'unsaturatedFatContent': '28 grams', 'carbohydrateContent': '29 grams', 'cholesterolContent': None, 'fatContent': '35 grams', 'fiberContent': '2 grams', 'proteinContent': '47 grams', 'saturatedFatContent': '4 grams', 'sodiumContent': '684 milligrams', 'sugarContent': '1 gram', 'transFatContent': '0 grams'}
easy, quick, main course

['6 pounds very fresh striped bass or bluefish', '2 cups flour', '1 tablespoon kosher salt', '1/4 cup freshly cracked black pepper', '1/4 cup Old Bay spice mix', '6 eggs', '1/2 cup half-and-half', '3 cups panko', '1 1/4 to 1 1/2 cups vegetable oil of your choice']
Cut fish into chunks about 2 inches square; you should have about 24 chunks.


Test out a few recipes first

In [11]:
from nytc import NYTC

recipes = pd.Series(recipe_urls, name='url')
features = ['recipe_name', 'nutrition', 'category', 'cuisine', 'ingredient', 'instruction']

feature_df = pd.DataFrame(columns=features)
for i in tqdm(range(10)):
    url = recipes.iloc[i]
    try:
        recipe = NYTC(url)
        feature_df.loc[i] = [getattr(recipe, feature)() for feature in features]
        time.sleep(np.random.uniform(0.05, 1))
    except:
        feature_df.loc[i] = [np.nan] * len(features)


  0%|          | 0/10 [00:00<?, ?it/s]

In [12]:
feature_df.head(3)

,recipe_name,nutrition,category,cuisine,ingredient,instruction
0,Crispy Potato Tacos,"{'@context': 'http://schema.org', '@type': 'NutritionInformation', 'calories': 656, 'unsaturatedFatContent': '14 grams', 'carbohydrateContent': '86 grams', 'cholesterolContent': None, 'fatContent': '27 grams', 'fiberContent': '12 grams', 'proteinContent': '22 grams', 'saturatedFatContent': '10 grams', 'sodiumContent': '1201 milligrams', 'sugarContent': '8 grams', 'transFatContent': '0 grams'}","dinner, tacos, appetizer, main course",mexican,"[Sea salt, 1 1/2 pounds potatoes (any variety), scrubbed and cut into 1 1/2-inch pieces, 1 1/2 cups grated Cheddar, Handful of cilantro, leaves and stems finely chopped, 1 small garlic clove, finely chopped, 1 teaspoon ground cumin, 1 teaspoon paprika, 16 to 18 corn tortillas, Neutral oil, as needed, Any combination of sliced lettuce or cabbage, very finely sliced red onion or sour cream (all optional), for serving, 3 tomatoes (about 1 pound), chopped, 1/2 red onion, roughly chopped, Small handful of cilantro, leaves and stems roughly chopped, 1 fresh serrano or Fresno chile (seeded, if you prefer less spice), 1 garlic clove, chopped, 1 teaspoon ground cumin, 1 teaspoon dried oregano, 1 teaspoon granulated sugar, Sea salt, 3/4 cup vegetable stock]","[Bring a large saucepan of salted water to a boil. Add the potatoes and cook for 15 to 20 minutes, until tender. (Check them by inserting a fork or knife into the largest potato piece. If it goes in and out easily, the potato is ready.) Drain and allow to cool for a few minutes., Make the spicy red salsa: Place tomatoes, onion, cilantro, chile, garlic, cumin, oregano, sugar and 1 teaspoon salt into a blender or food processor and blitz until completely smooth. Pour the purée into a saucepan, add the vegetable stock and bring to a boil. Reduce the heat to low and simmer for 15 to 20 minutes until darker in color and slightly thickened, while you prepare the remaining ingredients., Place the cooled potatoes in a bowl and roughly mash them. (It does not have to be smooth; a chunky texture is great.) Add the Cheddar, cilantro, garlic, cumin, paprika and 1 teaspoon sea salt and mix to combine., Place a large skillet over medium-high heat and, working in batches, add the corn tortillas and heat until soft and pliable. Remove from the pan and cover the tortillas with a clean kitchen towel to keep them warm. Fill each warmed tortilla with 1 to 2 tablespoons of the potato mixture, then fold in half and press down lightly., In the same skillet, add enough oil to cover the bottom of the pan and warm over medium-high heat. Place three or four tacos in the oil, pressing down lightly with a spatula so that the edges are in the oil, and fry for 1 to 2 minutes, until golden and crispy. Flip them over and repeat on the other side. Repeat with the remaining tacos., Serve the tacos with the spicy red salsa and any of the optional serving suggestions. (The potatoes can be cooked and mashed 2 days ahead and stored in an airtight container in the fridge. The salsa can be made 2 days ahead and kept in the fridge. For freezing info, see Tip.)]"
1,Cashew Celery,"{'@context': 'http://schema.org', '@type': 'NutritionInformation', 'calories': 299, 'unsaturatedFatContent': '15 grams', 'carbohydrateContent': '19 grams', 'cholesterolContent': None, 'fatContent': '21 grams', 'fiberContent': '5 grams', 'proteinContent': '12 grams', 'saturatedFatContent': '3 grams', 'sodiumContent': '507 milligrams', 'sugarContent': '4 grams', 'transFatContent': '0 grams'}","dinner, easy, quick, weeknight, vegetables, main course",,"[2 teaspoons cornstarch, 1/4 cup vegetable stock or water, 2 1/2 tablespoons Shaoxing rice wine, 4 teaspoons soy sauce or tamari, Neutral oil, 4 to 6 celery stalks (about 8 ounces), trimmed and sliced diagonally, 1 (1-inch) piece fresh ginger, peeled and finely chopped, 2 garlic cloves, finely chopped, 3 scallions, trimmed and sliced, 6 ounces dark, leafy greens (such as gai lan, c

In [13]:
feature_df = pd.DataFrame(columns=features)
for i in tqdm(range(len(recipe_urls))):
    url = recipes.iloc[i]
    try:
        recipe = NYTC(url)
        feature_df.loc[i] = [getattr(recipe, feature)() for feature in features]
        time.sleep(np.random.uniform(0.05, 1))
    except:
        feature_df.loc[i] = [np.nan] * len(features)

  0%|          | 0/22830 [00:00<?, ?it/s]

In [14]:
data_df = pd.concat([recipes, feature_df], axis=1)

In [15]:
data_df.to_pickle('../data/nytc_data.pkl')

Sanity Check

In [17]:
data_df.sample(10)


,url,recipe_name,nutrition,category,cuisine,ingredient,instruction
12607,https://cooking.nytimes.com/recipes/11581-caramel-sauce,Caramel Sauce,"{'@context': 'http://schema.org', '@type': 'NutritionInformation', 'calories': 223, 'unsaturatedFatContent': '3 grams', 'carbohydrateContent': '35 grams', 'cholesterolContent': None, 'fatContent': '10 grams', 'fiberContent': '0 grams', 'proteinContent': '1 gram', 'saturatedFatContent': '6 grams', 'sodiumContent': '11 milligrams', 'sugarContent': '34 grams', 'transFatContent': None}",dessert,,"[1 cup sugar, 2/3 cup heavy cream, warmed, 1/4 to 1/3 cup fresh lime juice]","[Melt the sugar in a heavy-bottomed 2 1/2-quart pot over high heat, cooking until sugar melts and turns a light amber color, about 3 minutes. Watch carefully so that sugar does not burn., Stir cream in slowly until the mixture is combined. It will bubble up. Remove from heat., Stir in juice, adjusting to taste. If sauce is not used immediately, cover and refrigerate. This can be made up to 2 days ahead. To serve, reheat and serve warm, not hot.]"
8505,https://cooking.nytimes.com/recipes/1014600-tropical-pineapple-sauce,Tropical Pineapple Sauce,"{'@context': 'http://schema.org', '@type': 'NutritionInformation', 'calories': 363, 'unsaturatedFatContent': '0 grams', 'carbohydrateContent': '90 grams', 'cholesterolContent': None, 'fatContent': '0 grams', 'fiberContent': '3 grams', 'proteinContent': '1 gram', 'saturatedFatContent': '0 grams', 'sodiumContent': '7 milligrams', 'sugarContent': '84 grams', 'transFatContent': None}","condiments, project, sauces and gravies",,"[8 cups finely diced, very ripe pineapple; about 2 pineapples, 2 cups white sugar, 1 cup brown sugar, 6 stalks fresh mint, rinsed and tied into a bundle, Juice of one lemon, 1/4 cup dark rum]","[Dice pineapple into 1/2-inch pieces, removing any brown eyes. Combine fruit, sugars, mint and lemon juice. Cover and leave on the counter for 1 hour., Stir pineapple mixture and discard the mint. Add mixture to a heavy nonreactive pan and simmer slowly, stirring regularly, for about an hour, until sauce is very thick and clear and has reached 220 degrees., Add rum. Stir until mixture is thick and clear again, about 5 minutes. Sauce will keep 3 months in the refrigerator or a year if processed., To process, put a rack in a large stockpot or line the pot with a folded kitchen towel, then fill the pot with water and bring to a boil. Add 4 half-pint canning jars and boil for 10 minutes. Jars may be left in the warm water in the pot until ready to be filled. Alternatively, sterilize the jars by running them through a dishwasher cycle, leaving them inside until ready to fill., Place canning rings in a small saucepan, cover with water and bring to a boil. Turn off heat and add the lids to soften rubber gaskets. Rings and lids may be left in the water until jars are filled., Remove the jars from the water bath and ladle hot sauce into warm jars, leaving 1/2-inch of headspace. Run a plastic knife gently around the inside of the jar to remove any air bubbles. Recheck the headspace., Wipe jar rims clean with a damp towel. Place the lids on the jars, screw on the rings and lower the jars back into the pot of boiling water. Return to a full boil and boil for 15 minutes. Transfer jars to a folded towel and let cool for 12 hours. You should hear them ping as they seal., Once cool, test the seals by removing rings and lifting jars by the flat lids. If a lid releases, the seal has not formed. Unsealed jars should be refrigerated and used within 3 months, or reprocessed. Rings and jars may be reused, but a new flat lid must be used each time.]"
12340,https://cooking.nytimes.com/recipes/8884-ginger-semifreddo,Ginger Semifreddo,NaN,dessert,italian,"[1 large egg, 4 large egg yolks, Scant 1/2 cup acacia, orange blossom or other honey, 2 tablespoons finely grated fresh ginger, 1 1/3 cups heavy cream, 2 to 3 nuggets preserved ginger in syrup (available at specialty food markets), finely diced]